In [1]:
import os
import base64
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()


endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

pprint(endpoint)
pprint(deployment)
pprint(subscription_key)

'https://azure-openai-7ai-050-3.openai.azure.com/'
'gpt-4o-mini'
'Dh3GiquzLWWMbG96CwwXVgLB766y5f5itgT7iru35ib28gsw5sQKJQQJ99BFACYeBjFXJ3w3AAABACOG5mnU'


In [ ]:
from openai import AzureOpenAI

# 키 기반 인증을 사용하여 Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# 채팅 프롬프트 준비
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
            }
        ],
    }
]

# 음성이 사용되는 경우 음성 결과 포함
messages = chat_prompt

while True:
    user_input = input("Enter your message(quit to exit): ")
    if user_input.lower() == "quit":
        break
    if len(user_input) == 0:
        print("Please enter a message.")
        continue

    messages.append({"role": "user", "content": user_input})

    # 완료 생성
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": response})

    messages = [messages[0]] + messages[min(len(messages) - 1, 10) * -1:]
    pprint(messages, compact=True)

    pprint(completion.choices[0].message.content)

In [ ]:
whisper_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_WHISPER_API_KEY"),
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_WHISPER_ENDPOINT_URL"),
)

whisper_deployment_id = os.getenv(
    "AZURE_OPENAI_WHISPER_DEPLOYMENT"
)  # This will correspond to the custom name you chose for your deployment when you deployed a model."

audio_test_file = "./recording_first.wav"

result = whisper_client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"), model=whisper_deployment_id, language="ko", prompt="입력된 음성은 한국어야, 한국어로 출력해줘"
)

# 채팅 프롬프트 준비
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
            }
        ],
    }
]

messages = chat_prompt + [{"role": "user", "content": result.text}]

completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
)

pprint(completion.choices[0].message.content)

In [32]:
from datetime import datetime
import pytz
from pprint import pprint

print(datetime.now())


def get_current_time(timezone="Asia/Seoul"):
    # pprint(f"timezone: {timezone}, current time: {datetime.now(pytz.timezone(timezone)).strftime('%Y-%m-%d %H:%M:%S')}")
    return datetime.now(pytz.timezone(timezone)).strftime("%Y-%m-%d %H:%M:%S")


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time using timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone to get the current time in",
                    }
                },
                # "required": ["timezone"],
            },
        },
    },
]

2025-06-23 17:32:26.705442


In [17]:
from openai import AzureOpenAI
import json

def request_to_openai(messages, tools=None):
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools=tools,
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    return completion.choices[0].message

# 키 기반 인증을 사용하여 Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# 채팅 프롬프트 준비
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "사용자가 정보를 찾는 데 도움이 되는 AI professional assistant",
            }
        ],
    }
]

messages = chat_prompt

print("챗봇 시작! 'quit'를 입력하여 종료하세요.")
while True:
    user_input = input("메시지를 입력하세요 (quit로 종료): ")
    if user_input.lower() == "quit":
        print("챗봇을 종료합니다.")
        break
    if len(user_input) == 0:
        print("메시지를 입력해주세요.")
        continue # 입력이 없으면 다시 프롬프트

    pprint(f"사용자: {user_input}")
    messages.append({"role": "user", "content": user_input})

    # 첫 번째 완료 생성 (도구 호출 여부 확인)
    response = request_to_openai(messages, tools=tools)
    # pprint(response, compact=True)

    # 응답에 tool_calls가 있는지 확인
    if response.tool_calls:
        print("--- Tool Call 감지 ---")
        for tool_call in response.tool_calls:
            tool_id = tool_call.id
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            print(f"  ID: {tool_id}")
            print(f"  Name: {tool_name}")
            print(f"  Args: {tool_args}")

            # 정의된 도구 함수에 따라 호출 및 결과 메시지 생성
            if tool_name == "get_current_time":
                function_response_content = get_current_time(
                    timezone=tool_args.get("timezone", "Asia/Seoul") # 기본값 서울로 설정
                )
                print(f"  함수 실행 결과: {function_response_content}")
                messages.append(
                    {
                        "tool_call_id": tool_id,
                        "role": "function", # role을 'function'으로 유지
                        "name": tool_name,
                        "content": function_response_content,
                    }
                )
            else:
                # 정의되지 않은 도구 호출 처리
                error_message = json.dumps({"error": f"Unknown tool: {tool_name}"})
                print(f"  오류: 알 수 없는 도구 호출: {tool_name}")
                messages.append(
                    {
                        "tool_call_id": tool_id,
                        "role": "function", # role을 'function'으로 유지
                        "name": tool_name,
                        "content": error_message,
                    }
                )

        # 도구 호출 결과를 모델에 다시 전달하여 최종 응답 생성
        # 중요한 것은 이 시점에서 messages 리스트에 'role': 'function' 메시지가 추가된 상태라는 점입니다.
        pprint(messages[-1], compact=True)

        # OpenAI의 Chat Completions API를 사용할 때, tool call(도구 호출)이 발생하면
        # 모델은 'function' 역할의 메시지(즉, 도구 실행 결과)를 받기 전까진 최종 답변을 생성하지 않습니다.
        # 도구 실행 결과를 'function' 메시지로 추가하면, 모델이 바로 자연어 답변을 반환할 것 같지만,
        # 실제로는 finish_reason이 'tool_calls'로 끝나면서 content가 None인 경우가 많습니다.
        # 이는 OpenAI가 tool call 프로토콜을 설계할 때, 도구 실행 결과를 받은 후
        # "이제 그 결과를 사용자에게 설명해달라"는 명확한 사용자 프롬프트가 한 번 더 필요하도록 설계했기 때문입니다.
        # 즉, function 메시지 이후에 추가로 user 역할의 메시지(예: "주어진 결과를 출력해주세요.")를 넣어주면
        # 그제서야 모델이 도구 실행 결과를 바탕으로 자연어 답변을 생성합니다.
        # 
        # 만약 이 단계를 생략하면, 모델이 도구 호출만 반복하거나, content가 비어있는 응답을 반환할 수 있습니다.
        # 
        # 관련 참고: https://community.openai.com/t/chat-completion-ai-returning-content-as-none-and-finish-reason-as-tool-call/1151616
        messages.append({"role": "user", "content": "주어진 결과를 출력해주세요."})
        response = request_to_openai(messages, tools=tools) # tools를 다시 전달하는 것이 중요
        pprint(response, compact=True) # 재호출된 응답도 출력

    messages.append(response) # 최종 응답을 메시지 기록에 추가
    pprint(f"AI: {response.content}")

챗봇 시작! 'quit'를 입력하여 종료하세요.
'사용자: 서울의 시간을 출력해줘'
--- Tool Call 감지 ---
  ID: call_IIcMA3FMn7KVsgwxOReCqh6w
  Name: get_current_time
  Args: {'timezone': 'Asia/Seoul'}
  함수 실행 결과: 2025-06-23 16:28:04
{'content': '2025-06-23 16:28:04',
 'name': 'get_current_time',
 'role': 'tool',
 'tool_call_id': 'call_IIcMA3FMn7KVsgwxOReCqh6w'}


BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

In [ ]:
import gradio as gr

def hello(name):
    return f"Hello {name}!"

with gr.Blocks() as demo:
    name_textbox = gr.Textbox(label="이름")
    output_textbox = gr.Textbox(label="출력")

    submit_button = gr.Button("제출")

    submit_button.click(fn=hello, inputs=name_textbox, outputs=output_textbox)

demo.launch()

In [19]:
import gradio as gr
import random

def click_send(message, chats):
    bot_message = random.choice(["안녕하세요!", "반가워요!", "좋은 하루 보내세요!"])

    chats.append({"role": "user", "content": message})
    chats.append({"role": "assistant", "content": bot_message})

    return "", chats

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="채팅", type="messages")

    input_textbox = gr.Textbox(label="")
    submit_button = gr.Button("전송")

    submit_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[input_textbox, chatbot])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [41]:
import gradio as gr

def click_send(message, chats):
    chats.append({"role": "user", "content": message})
    pprint(f"사용자: {message}")

    response = request_to_openai(chats, tools=tools)

    inner_chats = chats.copy()
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool_id = tool_call.id
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            # 정의된 도구 함수에 따라 호출 및 결과 메시지 생성
            if tool_name == "get_current_time":
                function_response_content = get_current_time(
                    timezone=tool_args.get(
                        "timezone", "Asia/Seoul"
                    )  # 기본값 서울로 설정
                )
                inner_chats.append(
                    {
                        "tool_call_id": tool_id,
                        "role": "function",  # role을 'function'으로 유지
                        "name": tool_name,
                        "content": function_response_content,
                    }
                )

        inner_chats.append({"role": "user", "content": "주어진 결과를 출력해주세요."})
        pprint(f"사용자: {inner_chats}", compact=True)
        response = request_to_openai(inner_chats, tools=tools)
    
    pprint(f"AI: {response.content}", compact=True)

    chats.append({"role": "assistant", "content": response.content})

    return "", chats


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="채팅", type="messages")

    input_textbox = gr.Textbox(label="")
    submit_button = gr.Button("전송")

    submit_button.click(
        fn=click_send, inputs=[input_textbox, chatbot], outputs=[input_textbox, chatbot]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


'사용자: 뉴욕 도쿄 서울의 시간을 순서대로 출력핫에ㅛ'
("사용자: [{'role': 'user', 'content': '뉴욕 도쿄 서울의 시간을 순서대로 출력핫에ㅛ'}, "
 "{'tool_call_id': 'call_qZKt1dNNd9hLKVkZvhWHghsz', 'role': 'function', "
 "'name': 'get_current_time', 'content': '2025-06-23 04:41:03'}, "
 "{'tool_call_id': 'call_6oGzX615CtMW13EDFQ0Qlso8', 'role': 'function', "
 "'name': 'get_current_time', 'content': '2025-06-23 17:41:03'}, "
 "{'tool_call_id': 'call_TPBxvZRKS4LtBcDHqLfa3xqR', 'role': 'function', "
 "'name': 'get_current_time', 'content': '2025-06-23 17:41:03'}, {'role': "
 "'user', 'content': '주어진 결과를 출력해주세요.'}]")
('AI: 주어진 결과는 다음과 같습니다:\n'
 '\n'
 '- 뉴욕: 2025-06-23 04:41:03\n'
 '- 도쿄: 2025-06-23 17:41:03\n'
 '- 서울: 2025-06-23 17:41:03')
'사용자: 도쿄 서울 뉴욕의 시간을 순서대로 출력하세요'
("사용자: [{'role': 'user', 'metadata': None, 'content': '뉴욕 도쿄 서울의 시간을 순서대로 "
 "출력핫에ㅛ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "
 "'주어진 결과는 다음과 같습니다:\\n\\n- 뉴욕: 2025-06-23 04:41:03\\n- 도쿄: 2025-06-23 "
 "17:41:03\\n- 서울: 2025-06-23 17:41:03'

Traceback (most recent call last):
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2201, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1983, in postprocess_data
    predict

'사용자: 파리의 시간을 알려줘'
("사용자: [{'role': 'user', 'metadata': None, 'content': '뉴욕 도쿄 서울의 시간을 순서대로 "
 "출력핫에ㅛ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "
 "'주어진 결과는 다음과 같습니다:\\n\\n- 뉴욕: 2025-06-23 04:41:03\\n- 도쿄: 2025-06-23 "
 "17:41:03\\n- 서울: 2025-06-23 17:41:03', 'options': None}, {'role': 'user', "
 "'metadata': None, 'content': '도쿄 서울 뉴욕의 시간을 순서대로 출력하세요', 'options': None}, "
 "{'role': 'assistant', 'metadata': None, 'content': '주어진 결과는 다음과 같습니다:\\n\\n- "
 '도쿄: 2025-06-23 17:41:19\\n- 서울: 2025-06-23 17:41:19\\n- 뉴욕: 2025-06-23 '
 "04:41:19', 'options': None}, {'role': 'user', 'metadata': None, 'content': "
 "'\\x08너가 사용할 수 있는 툴은 뭐야?', 'options': None}, {'role': 'assistant', "
 "'metadata': None, 'content': '제가 사용할 수 있는 툴은 현재 시간 정보를 가져오는 기능입니다. 이 기능을 "
 "사용하여 특정 시간대의 현재 시간을 조회할 수 있습니다. 예를 들어, 뉴욕, 도쿄, 서울의 현재 시간을 얻을 수 있습니다.', "
 "'options': None}, {'role': 'user', 'content': '파리의 시간을 알려줘'}, "
 "{'tool_call_id': 'call_8cRUP9vSDMBLYh4qz5OUgFu2', 'role': 'function'

Traceback (most recent call last):
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2201, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/j1star/Desktop/MS AI Projects/microsoft-ai-school/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1983, in postprocess_data
    predict

'사용자: 너에 대해 상세히 설명해줘'
('AI: 저는 AI 언어 모델로, 다양한 질문에 답변하고 정보를 제공하는 데 도움을 주기 위해 설계되었습니다. 제가 할 수 있는 일들은 '
 '다음과 같습니다:\n'
 '\n'
 '1. **정보 제공**: 다양한 주제에 대한 정보나 설명을 제공할 수 있습니다. 예를 들어, 과학, 역사, 기술, 문화 등 여러 분야에 '
 '걸쳐 질문에 답할 수 있습니다.\n'
 '\n'
 '2. **문서 작성**: 이메일, 보고서, 에세이 등 다양한 형식의 문서를 작성하는 데 도움을 줄 수 있습니다.\n'
 '\n'
 '3. **언어 번역**: 여러 언어 간의 번역을 도와줄 수 있습니다.\n'
 '\n'
 '4. **코드 작성 및 설명**: 프로그래밍 관련 질문에 답변하고, 코드 예제를 제공하거나 문제를 해결하는 데 도움을 줄 수 있습니다.\n'
 '\n'
 '5. **일상적인 질문 응답**: 일반적인 질문, 조언, 추천 등을 제공할 수 있습니다.\n'
 '\n'
 '6. **현재 시간 조회**: 특정 시간대의 현재 시간을 조회하는 기능을 가지고 있어, 이를 통해 시간 정보를 제공할 수 있습니다.\n'
 '\n'
 '저는 지속적으로 학습하고 있으며, 사용자와의 상호작용을 통해 더 나은 서비스를 제공하기 위해 노력하고 있습니다.')
'사용자: 파리의 시간을 알려줘'
("사용자: [{'role': 'user', 'metadata': None, 'content': '뉴욕 도쿄 서울의 시간을 순서대로 "
 "출력핫에ㅛ', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "
 "'주어진 결과는 다음과 같습니다:\\n\\n- 뉴욕: 2025-06-23 04:41:03\\n- 도쿄: 2025-06-23 "
 "17:41:03\\n- 서울: 2025-06-23 17:41:03', 'options': None}, {'role': 'user', "
 "'metadata': None

In [6]:
import os

import gradio as gr
from io import BytesIO
from PIL import Image
from base64 import b64decode

from openai import AzureOpenAI
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()


def show_image(image):
    return image


def generate_image(prompt):
    dalle_client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_DALLE_API_KEY"),
        api_version="2024-04-01-preview",
        azure_endpoint=os.getenv("AZURE_OPENAI_DALLE_ENDPOINT_URL"),
    )

    response = dalle_client.images.generate(
        model=os.getenv("AZURE_OPENAI_DALLE_DEPLOYMENT"),
        prompt=prompt,
        n=1,
        size="1024x1024",
        response_format="b64_json"
    )

    return Image.open(BytesIO(b64decode(response.data[0].b64_json)))


with gr.Blocks() as demo:
    input_textbox = gr.Textbox(label="이미지 생성 프롬프트")

    output_image = gr.Image(label="생성 이미지")

    submit_button = gr.Button("전송")

    submit_button.click(
        fn=generate_image, inputs=[input_textbox], outputs=[output_image]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [8]:
import os

from duckduckgo_search import DDGS
from dotenv import load_dotenv
load_dotenv()

query = "cat"

with DDGS() as ddgs:
    results = [result for result in ddgs.text(query, max_results=10)]

    pprint(results)

[{'body': 'The cat (Felis catus), also referred to as the domestic cat or '
          'house cat, is a small domesticated carnivorous mammal. It is the '
          'only domesticated species of the family Felidae.Advances in '
          'archaeology and genetics have shown that the domestication of the '
          'cat occurred in the Near East around 7500 BC. It is commonly kept '
          'as a pet and working cat, but also ranges freely as a feral cat '
          'avoiding ...',
  'href': 'https://en.wikipedia.org/wiki/Cat',
  'title': 'Cat - Wikipedia'},
 {'body': 'cat, (Felis catus), domesticated member (felid) of the family '
          'Felidae.The family is generally divided between cats from the '
          'subfamily Pantherinae, which roar (including lions, tigers, and '
          'leopards), and cats from the subfamily Felinae, which purr instead '
          '(including cougars, bobcats, and domestic cats).According to the '
          'latest research, purring likely stems 